# Data Parsing, Cleansing and Integration
## Task 3
#### Author Name: SUNITA VERMA


Date: 8/09/2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy


## Introduction
In Task 3, the main objective was to integrate two job advertisement datasets, namely dataset1_solution.csv and dataset2.csv, which were obtained from different sources. The integration process involved addressing schema conflicts, merging data, and resolving data-level conflicts to create one unified dataset.






##  Import libraries

In [241]:
# Code to import libraries as you need in this assessment
import re
import datetime
import difflib
import numpy as np
import pandas as pd
from IPython.display import display

### 1. Examining and loading data
Examine "dataset2.csv" the structure and schema.

In [242]:
# read in two data set

df1 = pd.read_csv('s3961780_dataset1_solution.csv',parse_dates=['OpenDate', 'CloseDate'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
df2 = pd.read_csv('s3961780_dataset2.csv')


In [243]:
# Display two dataframes together to compare
print('S1:' + str(df1.shape))
display(df1.head())


S1:(49251, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,69537180,Head of Business Assurance,Uk,Interaction Finance,non-specified,permanent,Accounting & Finance Jobs,47500.0,2013-01-31 12:00:00,2013-03-02 12:00:00,cv-library.co.uk
1,68216910,Machine Tool Wiremen / Panel Wiremen,Bradford,Network Employment Consultancy,non-specified,permanent,Engineering Jobs,23040.0,2013-03-14 15:00:00,2013-06-12 15:00:00,cv-library.co.uk
2,70166734,sales (no cold calling),Stoke-on-trent,Oakland House Recruitment,non-specified,permanent,Sales Jobs,15500.0,2012-05-27 00:00:00,2012-06-26 00:00:00,cv-library.co.uk
3,70763600,Nurse Medical Advisor PART TIME,Guildford,Reed Health,part_time,non-specified,Healthcare & Nursing Jobs,31000.0,2013-08-29 12:00:00,2013-09-12 12:00:00,cv-library.co.uk
4,69559871,Sales/Customer Service Engineer,Uk,Grad Central,non-specified,permanent,Engineering Jobs,19500.0,2012-05-13 12:00:00,2012-08-11 12:00:00,cv-library.co.uk


In [244]:
print('S2:' + str(df2.shape))
display(df2.head())

S2:(5000, 9)


,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [245]:
# Code to inspect data and identify conflicts in schema
print("--------------------For S1 data--------------------")
print (df1.info())
print("---------------------------------------------------")
print("--------------------For S2 data--------------------")
print (df2.info())
print("---------------------------------------------------")

--------------------For S1 data--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49251 entries, 0 to 49250
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            49251 non-null  int64         
 1   Title         49251 non-null  object        
 2   Location      49251 non-null  object        
 3   Company       49251 non-null  object        
 4   ContractType  49251 non-null  object        
 5   ContractTime  49251 non-null  object        
 6   Category      49251 non-null  object        
 7   Salary        49251 non-null  float64       
 8   OpenDate      49251 non-null  datetime64[ns]
 9   CloseDate     49251 non-null  datetime64[ns]
 10  SourceName    49251 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 4.1+ MB
None
---------------------------------------------------
--------------------For S2 data--------------------

***Global Schema for Job Listings (S1):***

- Id: Unique identifier for each job listing (Integer).
- Title: Job title or description (String).
- Location: Location of the job (String).
- Company: Name of the company offering the job (String).
- ContractType: Type of contract (String, e.g., 'non-specified', 'part_time','permanent', etc.).
- ContractTime: Duration of the contract (String, e.g., 'non-specified', 'permanent', etc.).
- Category: Category of the job (String).
- Salary: Salary for the job (Float).
- OpenDate: Date and time when the job listing opened (datetime in the format 'yyyy-mm-dd hh:mm:ss').
- CloseDate: Date and time when the job listing closed (datetime in the format 'yyyy-mm-dd hh:mm:ss').
- SourceName: Source or platform where the job listing was posted (String).


***Global Schema for Job Postings (S2):***

- Monthly Payment: Monthly payment for the job posting (Float).
- Closing: Date and time when the job posting closes (String in the format 'yyyy-mm-dd hh:mm:ss').
- Organisation: Name of the organization offering the job (String).
- Location: Location of the job (String).
- Category: Category of the job (String).
- Full-Time Equivalent (FTE): Full-Time Equivalent value (Float).
- Opening: Date and time when the job posting opens (string in the format 'yyyy-mm-dd hh:mm:ss').
- Job Title: Title of the job (String).
- Type: Type of job (String).

### 2. Resolving schema conflicts
Here are some general information about the conflicts I found in two schemas and steps you taken to resolve them.

#### Conflict 1: Naming Conflict of Columns
A significant conflict between the global schemas of both datasets (S1 and S2) is the naming conflict of columns. Many columns in both datasets have similar or identical meanings but different names. This can lead to confusion when trying to combine or analyze data from both sources. For example: 'Monthly Payment' in S2 is 'Salary' in S1, 'Closing' in S2 is 'CloseDate' in S1, 'Organisation' in S2 is 'Company' in S1, 'Full-Time Equivalent (FTE)' in S2 is 'ContractType' in S1,'Opening' in S2 is 'OpenDate' in S1,Job Title' in S2 is 'Title' in S1,'Type' in S2 is 'ContractType' in S1.

To fix this conflict we use :

Column Renaming Method: Using the mapping dictionary, the column names in S2 were renamed to match the corresponding column names in S1. This renaming process was carried out using data manipulation tools such as pandas.

In [246]:
# Define a mapping dictionary for column renaming

column_mapping = {
    'Monthly Payment': 'Salary',
    'Closing': 'CloseDate',
    'Organisation': 'Company',
    'Location': 'Location',
    'Category': 'Category',
    'Full-Time Equivalent (FTE)': 'ContractType',
    'Opening': 'OpenDate',
    'Job Title': 'Title',
    'Type': 'ContractTime'
}

# Rename the columns
df2.rename(columns=column_mapping, inplace=True)

# Print the updated DataFrame
print(df2.head())


    Salary            CloseDate                    Company          Location  \
0  1200.00  2012-12-27 12:00:00         Protocol Education                UK   
1  3125.00  2013-08-25 15:00:00                Cordius Ltd                UK   
2  4333.33  2013-01-16 15:00:00  Understanding Recruitment  Walton-On-Thames   
3  3750.00  2013-02-13 15:00:00     Wallace Hind Selection                UK   
4  2916.67  2013-01-12 12:00:00        Greenhill Group Ltd   Eastern England   

                 Category  ContractType             OpenDate  \
0               Education           0.8  2012-11-27 12:00:00   
1  Information Technology           1.0  2013-06-26 15:00:00   
2  Information Technology           1.0  2012-10-18 15:00:00   
3                   Sales           1.0  2012-12-15 15:00:00   
4                   Sales           1.0  2012-12-29 12:00:00   

                                               Title ContractTime  
0             Higher Level Teaching Assistant (HLTA)          NaN 

In [247]:
# print column names
print(df2.columns)

Index(['Salary', 'CloseDate', 'Company', 'Location', 'Category',
       'ContractType', 'OpenDate', 'Title', 'ContractTime'],
      dtype='object')


#### Conflict 2: Data Type Mismatching

The "Closing" and Opening column in S2 is defined as a string, while the corresponding "CloseDate" and "OpenDate" column in S1 is defined as a datetime.In addition to that, 'ContractType'column in S1 is defined as a string while 'Full-Time Equivalent (FTE)'column in S2 is defined as a Float. This is a data type mismatch between the two schemas, and it may require data conversion during integration.
......

In [248]:
df2['CloseDate'] = pd.to_datetime(df2['CloseDate'], format='%Y-%m-%d %H:%M:%S')
df2['OpenDate'] = pd.to_datetime(df2['OpenDate'], format='%Y-%m-%d %H:%M:%S')


In [249]:
# Define a function to categorize values
def categorize_fte(value):
    if value > 0.4:
        return 'full_time'
    elif 0.1 <= value <= 0.4:
        return 'part_time'
    else:
        return 'non-specified'

# Create a new column "EmploymentType" based on the categorization
df2['ContractType'] = df2['ContractType'].apply(categorize_fte)

# Display the resulting DataFrame
print(df2)


       Salary           CloseDate                       Company  \
0     1200.00 2012-12-27 12:00:00            Protocol Education   
1     3125.00 2013-08-25 15:00:00                   Cordius Ltd   
2     4333.33 2013-01-16 15:00:00     Understanding Recruitment   
3     3750.00 2013-02-13 15:00:00        Wallace Hind Selection   
4     2916.67 2013-01-12 12:00:00           Greenhill Group Ltd   
...       ...                 ...                           ...   
4995  5833.33 2013-03-31 00:00:00  Jonathan Lee Recruitment Ltd   
4996  5000.00 2012-08-27 12:00:00                    Impact 360   
4997  2833.33 2013-07-23 00:00:00       Aspire Data Recruitment   
4998  4791.67 2012-12-18 15:00:00              Annapurna HR Ltd   
4999  3541.67 2013-10-20 12:00:00   Parallel Consulting Limited   

              Location                Category ContractType  \
0                   UK               Education    full_time   
1                   UK  Information Technology    full_time   
2     

In [250]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Salary        5000 non-null   float64       
 1   CloseDate     5000 non-null   datetime64[ns]
 2   Company       4514 non-null   object        
 3   Location      5000 non-null   object        
 4   Category      5000 non-null   object        
 5   ContractType  5000 non-null   object        
 6   OpenDate      5000 non-null   datetime64[ns]
 7   Title         5000 non-null   object        
 8   ContractTime  3580 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 351.7+ KB
None


#### Conflict 3: Data column Missing

We found that dataset S2 don not have 'Id' column, this can make be a hinderance in integration. Generating unique IDs for records in S2 is essential for data consistency, integrity, and integration. It ensures each record is distinguishable, simplifies data management, and aligns with global schema requirements.

In [251]:
#create a new 'Id' column for df2 using a unique identifier generator.
df2['Id'] = range(10000000, 10000000 + len(df2))
df2 = df2[['Id'] + [col for col in df2 if col != 'Id']]
df2.head()

,Id,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime
0,10000000,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,full_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,10000001,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,10000002,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,10000003,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,10000004,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,full_time,2012-12-29 12:00:00,Channel Account Manager Software,NaN


During the integration process, it was observed that the 'SourceName' column, which represents the source or platform where the job listing was posted, exists in the S1 dataset but is not present in the S2 dataset. To ensure consistency and alignment between the two datasets, and to simplify the integration workflow, the decision was made to drop the 'SourceName' column from the S1 dataset. This action eliminates the presence of a column that is not shared between the datasets and avoids potential conflicts or inconsistencies during data merging. The focus is on integrating relevant data fields that match the global schema and facilitate seamless data analysis.

In [252]:
# Drop the 'SourceName' column from S1
df1 = df1.drop(columns=['SourceName'])
df1.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate
0,69537180,Head of Business Assurance,Uk,Interaction Finance,non-specified,permanent,Accounting & Finance Jobs,47500.0,2013-01-31 12:00:00,2013-03-02 12:00:00
1,68216910,Machine Tool Wiremen / Panel Wiremen,Bradford,Network Employment Consultancy,non-specified,permanent,Engineering Jobs,23040.0,2013-03-14 15:00:00,2013-06-12 15:00:00
2,70166734,sales (no cold calling),Stoke-on-trent,Oakland House Recruitment,non-specified,permanent,Sales Jobs,15500.0,2012-05-27 00:00:00,2012-06-26 00:00:00
3,70763600,Nurse Medical Advisor PART TIME,Guildford,Reed Health,part_time,non-specified,Healthcare & Nursing Jobs,31000.0,2013-08-29 12:00:00,2013-09-12 12:00:00
4,69559871,Sales/Customer Service Engineer,Uk,Grad Central,non-specified,permanent,Engineering Jobs,19500.0,2012-05-13 12:00:00,2012-08-11 12:00:00


Mapping the categories from df2 to df1 involves making the category names in both datasets match. This helps in putting similar data together. For example, if df1 has "IT Jobs" and df2 has "Information Technology," mapping them means considering both as the same thing.

In [253]:
print("S1 dataset:\n",df1['Category'].value_counts())
print("\n-----------------------------------------------\n")
print("S2 dataset:\n",df2['Category'].value_counts())

S1 dataset:
 IT Jobs                             12718
Healthcare & Nursing Jobs            7773
Engineering Jobs                     7358
Accounting & Finance Jobs            6577
Sales Jobs                           4787
Hospitality & Catering Jobs          4210
Teaching Jobs                        3388
PR, Advertising & Marketing Jobs     2440
Name: Category, dtype: int64

-----------------------------------------------

S2 dataset:
 Information Technology    1317
Health                     789
Engineering                744
Finance                    668
Sales                      498
Hospitality                399
Education                  326
Marketing                  259
Name: Category, dtype: int64


In [254]:
# Mapping dictionary to map categories from df2 to df1
category_mapping = {
    'Information Technology': 'IT Jobs',
    'Health': 'Healthcare & Nursing Jobs',
    'Engineering': 'Engineering Jobs',
    'Finance': 'Accounting & Finance Jobs',
    'Sales': 'Sales Jobs',
    'Hospitality': 'Hospitality & Catering Jobs',
    'Education': 'Teaching Jobs',
    'Marketing': 'PR, Advertising & Marketing Jobs'
}

# Apply the mapping to df2
df2['Category'] = df2['Category'].map(category_mapping)
df2.head()

,Id,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime
0,10000000,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Teaching Jobs,full_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,10000001,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,IT Jobs,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,10000002,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,IT Jobs,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,10000003,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales Jobs,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,10000004,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales Jobs,full_time,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [255]:
print("S1 dataset:\n",df1['Location'].value_counts())
print("\n-----------------------------------------------\n")
print("S2 dataset:\n",df2['Location'].value_counts())

S1 dataset:
 Uk                   7523
London               6333
South East London    2601
The City             1096
Central London        799
                     ... 
Cwmbran                 8
Twickenham              8
Waterlooville           8
North Finchley          7
Thorpe St. Andrew       7
Name: Location, Length: 479, dtype: int64

-----------------------------------------------

S2 dataset:
 UK                   761
London               644
South East London    311
The City             125
Central London        72
                    ... 
Haslemere              1
Worksop                1
West Malling           1
Hove                   1
Tewkesbury             1
Name: Location, Length: 437, dtype: int64


In [256]:
# Define a function to convert the entire "Location" column to lowercase
def LowercaseLocation(string_val):
    return string_val.lower()

# apply the function to the "Location" column
df2['Location'] = df2['Location'].apply(LowercaseLocation)

# Define a function to capitalize the first letter of each word
def CapitalizingLocation(string_val):
    words = string_val.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# apply the function to the "Location" column
df2['Location'] = df2['Location'].apply(CapitalizingLocation)


In [257]:
print(difflib.get_close_matches('Oxfords',set(df2['Location']).difference(['Oxfords']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('Haslemere',set(df2['Location']).difference(['Haslemere']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('Worksop',set(df2['Location']).difference(['Worksop']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('West Malling',set(df2['Location']).difference(['West Malling']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('Hove',set(df2['Location']).difference(['Hove']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('Haslemere',set(df2['Location']).difference(['Haslemere']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('Tewkesbury',set(df2['Location']).difference(['Tewkesbury']), n = 1,cutoff=0.85) )
print(difflib.get_close_matches('West Malling',set(df2['Location']).difference(['West Malling']), n = 1,cutoff=0.85) )

['Oxford']
[]
[]
[]
[]
[]
[]
[]


In [258]:
correction_dict = {'Oxfords':'Oxford'}
df2['Location'].replace(correction_dict,inplace=True)
df2['Location']= df2['Location'].astype(str)
df2['Location'].value_counts()

Uk                   761
London               644
South East London    311
The City             125
Central London        72
                    ... 
Haslemere              1
Worksop                1
West Malling           1
Hove                   1
Tewkesbury             1
Name: Location, Length: 437, dtype: int64

I have converted NaN values in the "ContractTime" column of S2 to "non-specified" to ensure alignment with the global schema, which requires a non-null value in this column. This move enhances data consistency and facilitates seamless integration with S1.

In [259]:
df2['ContractTime'].fillna('non-specified', inplace=True)

In [260]:
invalid_dates = df2['OpenDate'].isna()
invalid_dates

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: OpenDate, Length: 5000, dtype: bool

In [261]:
df2['ContractTime'] = df2['ContractTime'].apply(LowercaseLocation)
df2

,Id,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime
0,10000000,1200.00,2012-12-27 12:00:00,Protocol Education,Uk,Teaching Jobs,full_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),non-specified
1,10000001,3125.00,2013-08-25 15:00:00,Cordius Ltd,Uk,IT Jobs,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,permanent
2,10000002,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-on-thames,IT Jobs,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),permanent
3,10000003,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,Uk,Sales Jobs,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",non-specified
4,10000004,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales Jobs,full_time,2012-12-29 12:00:00,Channel Account Manager Software,non-specified
...,...,...,...,...,...,...,...,...,...,...
4995,10004995,5833.33,2013-03-31 00:00:00,Jonathan Lee Recruitment Ltd,London,Accounting & Finance Jobs,full_time,2013-01-30 00:00:00,Head of Regulatory Policy,permanent
4996,10004996,5000.00,2012-08-27 12:00:00,Impact 360,Berkshire,IT Jobs,full_time,2012-06-28 12:00:00,Android Developer eMarkets and eCommerce Ber...,permanent
4997,10004997,2833.33,2013-07-23 00:00:00,Aspire Data Recruitment,London,Accounting & Finance Jobs,full_time,2013-04-24 00:00:00,SAS Senior Collections Recoveries Analyst,permanent
4998,10004998,4791.67,2012-12-18 15:00:00,Annapurna HR Ltd,London,IT Jobs,full_time,2012-11-18 15:00:00,Lead PHP Magento Developer London ****k,permanent


### 3. Merging data

Here, we merge both are dataframe.

In [262]:
# Code to merge two data sets
df = pd.concat([df1,df2])
print (df.shape)
df.sample(10)

(54251, 10)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate
46935,69181399,Head of Incident and Recovery Management,Uk,The JM Group,non-specified,permanent,Accounting & Finance Jobs,60000.00,2012-03-24 12:00:00,2012-06-22 12:00:00
42583,68541156,Strategic Analysis and Planning Executive,Hertfordshire,INSTITUTION OF ENGINEERING AND TECHNOLOGY,full_time,permanent,"PR, Advertising & Marketing Jobs",35000.00,2013-04-03 12:00:00,2013-05-03 12:00:00
13308,66606203,Dynamics CRM Developer,Walsall,non-specified,non-specified,permanent,IT Jobs,37500.00,2013-03-09 00:00:00,2013-04-08 00:00:00
5105,69082915,Quality / QA Engineer,West Yorkshire,Kinetic PLC,non-specified,permanent,Engineering Jobs,25500.00,2012-06-20 12:00:00,2012-08-19 12:00:00
21997,68697230,Management Opportunities Cambridge,Uk,Sports Direct,non-specified,permanent,Sales Jobs,24000.00,2012-02-07 15:00:00,2012-05-07 15:00:00
18610,59104064,QA Manager – Northamptonshire **** CRJE****/...,Northamptonshire,Citizen Recruitment,non-specified,permanent,Engineering Jobs,38000.00,2012-08-28 00:00:00,2012-09-27 00:00:00
10739,71565448,Oracle Database Administrator / Oracle DBA Le...,Uk,Coal IT Services,non-specified,permanent,IT Jobs,37500.00,2013-03-27 00:00:00,2013-06-25 00:00:00
45922,66665813,"Fluent Italian, Reporting, Tax & TP Compliance...",London,Brewer Morris,full_time,contract,Accounting & Finance Jobs,87600.00,2013-08-24 15:00:00,2013-10-23 15:00:00
23517,71527180,Superb Sous Chef required **** Cruise Line,Uk,Excite Recruitment,non-specified,non-specified,Hospitality & Catering Jobs,26500.00,2012-02-05 15:00:00,2012-04-05 15:00:00
2995,10002995,User Interface / Experience (UI/UE) Designer /...,Bradford,Elevate Recruitment,full_time,permanent,IT Jobs,3333.33,2013-11-29 00:00:00,2013-12-29 00:00:00


### 4. Resolving data conflicts:
information about the data conflicts have been found in the unified table and how we resolve them.


#### Conflict 1: complete check for duplications


In [263]:
# Code to inspect data and identify conflicts in data
### check duplication
duplicates = df[df.duplicated(keep=False)] # showing all duplicated records
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 0 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate


In [264]:
# check duplication
cols = ['Id'] # this should uniquely identified a record, as the same bike could not be used by multiple trip started at the same time
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 0 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate


In [265]:

# Define the columns to check for 'non-specified'
columns_to_check1 = ['ContractType', 'Company', 'ContractTime', 'Salary']
# Create a condition to filter rows
condition1 = (df[columns_to_check1] != 'non-specified').all(axis=1)
# Apply the condition to filter rows
df = df[condition1]


# Define the columns to check for 'non-specified'
columns_to_check2 = ['ContractType', 'ContractTime']
# Create a condition to filter rows
condition2 = (df[columns_to_check2] != 'non-specified').all(axis=1)
# Apply the condition to filter rows
df = df[condition2]

# Print the resulting DataFrame
print(df.head())

            Id                                              Title    Location  \
141   72340026                      RGN Band 5 Bronzefield Prison          Uk   
349   54900518                 Primary Teachers Norfolk / Suffolk     Norfolk   
359   69933333                           RGN Band 5 West Midlands  Birmingham   
743   71554929  Part Time Accounts Senior (practice background...   Sevenoaks   
1678  71556408    Primary teachers wanted by award winning agency      London   

                        Company ContractType ContractTime  \
141   Medacs Healthcare Nursing    part_time     contract   
349            Engage Education    part_time     contract   
359   Medacs Healthcare Nursing    part_time     contract   
743    Prospect Recruitment Ltd    part_time     contract   
1678           Engage Education    part_time     contract   

                       Category   Salary            OpenDate  \
141   Healthcare & Nursing Jobs  44188.0 2013-02-22 00:00:00   
349               

In [266]:
# check duplication
cols = ['Title'] # this should uniquely identified a record, as the same bike could not be used by multiple trip started at the same time
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 70 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate
41400,66701562,Account Executive – B2B / Technology,London,JFL Search Selection,full_time,permanent,"PR, Advertising & Marketing Jobs",21000.00,2013-09-15 15:00:00,2013-10-15 15:00:00
1002,10001002,Account Executive – B2B / Technology,London,JFL Search Selection,full_time,permanent,"PR, Advertising & Marketing Jobs",1750.00,2013-09-15 15:00:00,2013-10-15 15:00:00
41371,68215152,Account Manager b2b tech/media/ents,Central London,Unicorn Jobs,full_time,permanent,"PR, Advertising & Marketing Jobs",27500.00,2012-09-14 00:00:00,2012-12-13 00:00:00
1636,10001636,Account Manager b2b tech/media/ents,Central London,Unicorn Jobs,full_time,permanent,"PR, Advertising & Marketing Jobs",2291.67,2012-09-14 00:00:00,2012-12-13 00:00:00
153,10000153,Accounts Receivable Team Leader,Uk,VanRath Accountancy,full_time,permanent,Accounting & Finance Jobs,1666.67,2013-03-08 12:00:00,2013-06-06 12:00:00
43620,69673555,Accounts Receivable Team Leader,Uk,VanRath Accountancy,full_time,permanent,Accounting & Finance Jobs,20000.00,2013-03-08 12:00:00,2013-06-06 12:00:00
813,10000813,CAD/CAM Engineer,Belfast,PRG,full_time,permanent,Engineering Jobs,2708.33,2012-06-18 12:00:00,2012-09-16 12:00:00
44111,67385343,CAD/CAM Engineer,Belfast,PRG,full_time,permanent,Engineering Jobs,32500.00,2012-06-18 12:00:00,2012-09-16 12:00:00
30540,65196329,Commercial Retail Accountant Hampshire,Hampshire,Albior Financial Recruitment,full_time,permanent,Accounting & Finance Jobs,35000.00,2013-04-22 00:00:00,2013-06-21 00:00:00
2118,10002118,Commercial Retail Accountant Hampshire,Hampshire,Albior Financial Recruitment,full_time,permanent,Accounting & Finance Jobs,2916.67,2013-04-22 00:00:00,2013-06-21 00:00:00


Tell me about how you fix the conflict here

In [267]:
# Code to fix the conflict
df = df.drop_duplicates(cols, keep='last')
df.shape

(9949, 10)

In [268]:
# check duplication
cols = ['Title','Company','Category','OpenDate','CloseDate'] # this should uniquely identified a record, as the same bike could not be used by multiple trip started at the same time
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 0 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate


In [269]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9949 entries, 141 to 4999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            9949 non-null   int64         
 1   Title         9949 non-null   object        
 2   Location      9949 non-null   object        
 3   Company       9748 non-null   object        
 4   ContractType  9949 non-null   object        
 5   ContractTime  9949 non-null   object        
 6   Category      9949 non-null   object        
 7   Salary        9949 non-null   float64       
 8   OpenDate      9949 non-null   datetime64[ns]
 9   CloseDate     9949 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 855.0+ KB


In [270]:
df.shape

(9949, 10)

In [271]:
df.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate
141,72340026,RGN Band 5 Bronzefield Prison,Uk,Medacs Healthcare Nursing,part_time,contract,Healthcare & Nursing Jobs,44188.0,2013-02-22 00:00:00,2013-03-24 00:00:00
349,54900518,Primary Teachers Norfolk / Suffolk,Norfolk,Engage Education,part_time,contract,Teaching Jobs,30000.0,2012-10-08 00:00:00,2012-12-07 00:00:00
359,69933333,RGN Band 5 West Midlands,Birmingham,Medacs Healthcare Nursing,part_time,contract,Healthcare & Nursing Jobs,34032.0,2013-01-23 00:00:00,2013-02-06 00:00:00
743,71554929,Part Time Accounts Senior (practice background...,Sevenoaks,Prospect Recruitment Ltd,part_time,contract,Accounting & Finance Jobs,25000.0,2012-01-13 00:00:00,2012-01-27 00:00:00
1678,71556408,Primary teachers wanted by award winning agency,London,Engage Education,part_time,contract,Teaching Jobs,29400.0,2012-05-30 12:00:00,2012-06-13 12:00:00


......

#### Finding global key for the data
Find a proper global key for the integrated data and give justification here.

In [272]:
# Code to identify global key
# Assuming you have integrated your data into a DataFrame called integrated_df

# Check if the 'Id' column is unique
if df['Id'].is_unique:
    global_key = 'Id'
    print("Global key identified:", global_key)
else:
    print("The 'Id' column is not unique. You may need to consider other columns.")


Global key identified: Id


A suitable global key for the integrated data could be a composite key consisting of "Id" (unique identifier for each job listing) and "SourceName" (indicating the dataset source, e.g., for 'S1' or 'S2'). This key offers a clear distinction between records from different sources while maintaining data integrity. It allows for easy tracking, merging, and referencing of records from both datasets, ensuring a seamless integration process.However, since the "Source" column is not present in one of the datasets (S2), it complicates the integration process. As a result, the decision was made to use the "Id" column as a simpler and more reliable unique identifier for each job listing. This "Id" column ensures that every record in the integrated dataset remains distinct, even when the source information is not available for some listings.

.......

### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- 'dataset_integrated.csv'

In [273]:
# code to save output data
df.to_csv('dataset_integrated.csv', index=False)

## Summary of the Task
In Task 3, we merged two job advertisement datasets, dataset1_solution.csv and dataset2.csv, to create a unified dataset. To resolve schema conflicts, we ensured that columns from both datasets followed a common structure. For example, we mapped "Salary" from dataset 1 to "Monthly Payment" in dataset 2, and "Company" in dataset 1 to "Organisation" in dataset 2.

We then addressed data-level conflicts, such as duplicate records, using Pandas. For instance, we identified and removed duplicate job listings based on unique keys.

As a global key for the integrated dataset, we used the "Id" column, ensuring each job listing had a unique identifier. For example, job ID 69537180 from dataset 1 and job ID 10000001 from dataset 2 were integrated using this key.

Finally, we produced the integrated dataset dataset_integrated.csv, which combined job data from both sources for further analysis.